# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Import Data

In [2]:
df = pd.read_csv('../data/model_data.csv', index_col = 'id').drop('Unnamed: 0', axis = 1)

In [3]:
df.columns

Index(['date', 'home_team', 'opponent', 'attend', 'temp',
       'day_night_categories', 'visiting_OPS+', 'visiting_ERA+',
       'visiting_win_percent', 'city_rivals_flag', 'weekend_flag',
       'home_opener_flag', 'holiday_flag', 'rainy_flag', 'cap', 'shirt',
       'fireworks', 'bobblehead', 'promotion_flag'],
      dtype='object')

In [4]:
df.head()

,date,home_team,opponent,attend,temp,day_night_categories,visiting_OPS+,visiting_ERA+,visiting_win_percent,city_rivals_flag,weekend_flag,home_opener_flag,holiday_flag,rainy_flag,cap,shirt,fireworks,bobblehead,promotion_flag
id,,,,,,,,,,,,,,,,,,,
1905,2012-04-04,Miami Marlins,St. Louis Cardinals,36601,79,non_sunday_night,107,103,0.543,0,0,1,0,0,NO,NO,NO,NO,0
1319,2012-04-05,Cincinnati Reds,Miami Marlins,42956,60,non_sunday_day,85,100,0.426,0,0,1,0,0,NO,NO,NO,NO,0
1412,2012-04-05,New York Mets,Atlanta Braves,42080,53,non_sunday_day,90,117,0.580,0,0,1,0,0,NO,NO,NO,NO,0
184,2012-04-05,Cleveland Indians,Toronto Blue Jays,43190,44,non_sunday_day,94,91,0.451,0,0,1,0,0,NO,NO,NO,NO,0
1993,2012-04-05,San Diego Padres,Los Angeles Dodgers,42941,69,non_sunday_night,91,114,0.531,0,0,1,0,0,NO,YES,NO,NO,1


# Linear Regression

### Model Building

In [5]:
# Import libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [6]:
# train and test data
# drop opponent to avoid overfitting to the results of one specific season
X = df.drop(['opponent', 'date', 'attend', 'promotion_flag'], axis = 1)
y = df['attend']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
X.columns

Index(['home_team', 'temp', 'day_night_categories', 'visiting_OPS+',
       'visiting_ERA+', 'visiting_win_percent', 'city_rivals_flag',
       'weekend_flag', 'home_opener_flag', 'holiday_flag', 'rainy_flag', 'cap',
       'shirt', 'fireworks', 'bobblehead'],
      dtype='object')

In [8]:
# data preprocessing
categorical_vars = ['home_team', 'day_night_categories', 'cap', 'shirt', 'fireworks', 'bobblehead']
quantitative_vars = ['temp', 'visiting_OPS+', 'city_rivals_flag',
       'weekend_flag', 'home_opener_flag', 'holiday_flag', 'visiting_ERA+', 'visiting_win_percent', 'rainy_flag']

# one-hot encode categorical variables
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='first'))])

# standardize quantitative avariables for more effective interpretation of coefficients
quantitative_transformer = Pipeline(steps=[('scaler', StandardScaler())])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_vars),
        ('quantitative', quantitative_transformer, quantitative_vars)
    ])

In [9]:
# Create model
lm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# train on training data
lm.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['home_team',
                                                   'day_night_categories',
                                                   'cap', 'shirt', 'fireworks',
                                                   'bobblehead']),
                                                 ('quantitative',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'visiting_OPS+',
                                                   'city_rivals_flag',
                                                   'weekend_fl

In [10]:
# Make predictions
y_pred = lm.predict(X_test)

### Model Evaluation

##### Performance

In [11]:
# Interpretation: the model's predictions are off by an average of 4,335. It explains 68.4% of the variance in the data.
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 4335.8022040961205
R-squared: 0.6837748736212521


##### Coefficients

In [12]:
# Extract encoded categorical features from the pipeline
categorical_feature_names = lm.named_steps['preprocessor'] \
    .transformers_[0][1] \
    .named_steps['onehot'] \
    .get_feature_names(input_features=categorical_vars)

# Create a list of all features
all_feature_names = list(categorical_feature_names) + quantitative_vars  # Assuming you have defined quantitative_vars

# Extract coefficients from the linear regression model
coefficients = lm.named_steps['regressor'].coef_

# Create a DataFrame to display coefficients and their names
coefficients_df = pd.DataFrame({'feature': all_feature_names, 'coefficient': coefficients})

# Display the coefficients in descending order
# coefficients_df.sort_values('coefficient', ascending = False)
coefficients_df.iloc[coefficients_df['coefficient'].abs().argsort()[::-1]]

,feature,coefficient
19,home_team_Philadelphia Phillies,18413.079458
17,home_team_New York Yankees,18020.482273
22,home_team_San Francisco Giants,16352.258095
26,home_team_Texas Rangers,15132.095787
12,home_team_Los Angeles Dodgers,14135.089239
24,home_team_St. Louis Cardinals,13661.724870
2,home_team_Boston Red Sox,11655.178453
11,home_team_Los Angeles Angels,11100.171004
8,home_team_Detroit Tigers,10901.160813
3,home_team_Chicago Cubs,8671.179276


As expected, number one predictor of attendance is the home team. We'll filter these out since there are no actionable insights here, other than observing which teams draw large crowds

In [15]:
coeffs_no_home_team = coefficients_df[~coefficients_df['feature'].str.startswith('home_team')]
coeffs_ordered = coeffs_no_home_team.iloc[coeffs_no_home_team['coefficient'].abs().argsort()[::-1]]
coeffs_ordered

,feature,coefficient
35,bobblehead_YES,4885.772808
39,weekend_flag,2600.894941
34,fireworks_YES,2476.377110
32,cap_YES,2403.514089
33,shirt_YES,2302.707658
30,day_night_categories_sunday_day,-1893.123750
40,home_opener_flag,1549.518026
31,day_night_categories_sunday_night,-1484.565112
29,day_night_categories_non_sunday_night,-1251.102868
37,visiting_OPS+,959.670205


**Takeaways:**

(1) Promotions are extremely important to improving attendance, with bobbleheads being the most effective

(2) Weekends, sunday night games, and home openers are the biggest draws

(3) Fans would rather see opposing teams that are good offensively, but they do not care how good they are at pitching or what their record is.

In [16]:
# re-run the model without rainy_flag, visiting_win_percent or visitng_ERA+ since their coefficients are quite low
# model performance slightly improves (MAE down 8, R-squared up 0.1%)
# only a marginal increase in performance, but worth it given the simpler model

X_new = X.drop(['rainy_flag', 'visiting_win_percent', 'visiting_ERA+'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

new_quantitative_vars = ['temp', 'visiting_OPS+', 'city_rivals_flag',
       'weekend_flag', 'home_opener_flag', 'holiday_flag']

new_preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_vars),
        ('quantitative', quantitative_transformer, new_quantitative_vars)
    ])

new_lm = Pipeline(steps=[
    ('preprocessor', new_preprocessor),
    ('regressor', LinearRegression())
])

new_lm.fit(X_train, y_train)

y_pred = new_lm.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 4327.969964882387
R-squared: 0.6850265075930619


In [17]:
# update variables accordingly
X = X_new
quantitative_vars = new_quantitative_vars
preprocessor = new_preprocessor

# Random Forest

In [18]:
# data preprocessing
# one hot encode, still going to standardize - doesn't affect predictions, but will help with feature importance analysis
# train and test data
from sklearn.ensemble import RandomForestRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', categorical_transformer, categorical_vars),
        ('quantitative', quantitative_transformer, quantitative_vars)
    ])
rf_regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt', 'log2']
}
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['home_team',
                                                                          'day_night_categories',
                                                                          'cap',
                                                                          'shirt',
                                                                          'fireworks',
                                                                          'bobblehead']),
                                                                        ('quantitative',


In [20]:
# Select the best model
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
best_rf_regressor = grid_search.best_estimator_

Best Hyperparameters: {'regressor__max_depth': None, 'regressor__max_features': 'auto', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 300}


In [21]:
# Generate predictions and evaluate results with the optimized random forest regression model
y_pred = best_rf_regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Absolute Error: 3704.0765226427297
R-squared: 0.724684177247088


# Feature Importance

### Gini Importance

In [22]:
preprocessor = best_rf_regressor.named_steps['preprocessor']
categorical_features = preprocessor.named_transformers_['categorical']['onehot'] \
.get_feature_names(input_features=categorical_vars)
numeric_features = preprocessor.transformers_[0][2]

In [23]:
categorical_features = best_rf_regressor.named_steps['preprocessor'] \
    .transformers_[0][1] \
    .named_steps['onehot'] \
    .get_feature_names(input_features=categorical_vars)

# Create a list of all features
all_feature_names = list(categorical_feature_names) + quantitative_vars 

In [24]:
# Calculate gini importance and display for all variables
feature_importances = best_rf_regressor.named_steps['regressor'].feature_importances_
gini_importance_df = pd.DataFrame({'feature': all_feature_names, 'gini_importance': feature_importances})
# Display the coefficients in descending order
gini_importance_df.iloc[gini_importance_df['gini_importance'].abs().argsort()[::-1]]

,feature,gini_importance
36,temp,0.093774
39,weekend_flag,0.081633
37,visiting_OPS+,0.071637
19,home_team_Philadelphia Phillies,0.068428
17,home_team_New York Yankees,0.065686
22,home_team_San Francisco Giants,0.051777
12,home_team_Los Angeles Dodgers,0.049121
26,home_team_Texas Rangers,0.049025
24,home_team_St. Louis Cardinals,0.044030
40,home_opener_flag,0.032235


In [25]:
# Filter out home team - this is not actionable information for individual teams since this variable is fixed
gini_no_home_team = gini_importance_df[~gini_importance_df['feature'].str.startswith('home_team')]
gini_ordered = gini_no_home_team.iloc[gini_no_home_team['gini_importance'].abs().argsort()[::-1]]
gini_ordered

,feature,gini_importance
36,temp,0.093774
39,weekend_flag,0.081633
37,visiting_OPS+,0.071637
40,home_opener_flag,0.032235
35,bobblehead_YES,0.010510
29,day_night_categories_non_sunday_night,0.010246
30,day_night_categories_sunday_day,0.007137
33,shirt_YES,0.005494
34,fireworks_YES,0.005327
41,holiday_flag,0.004620


### Shapley Values

In [27]:
# apply preprocessing to training data and initialize explainer to calculate shapley values
import shap
X_train_preprocessed = best_rf_regressor.named_steps['preprocessor'].transform(X_train)
X_train_preprocessed_dense = X_train_preprocessed.toarray()
explainer = shap.Explainer(best_rf_regressor.named_steps['regressor'], X_train_preprocessed_dense)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [28]:
shap_values = explainer.shap_values(X_train_preprocessed_dense, check_additivity = False)

 99%|===================| 1624/1634 [02:44<00:01]        

In [29]:
# Display shapley values by feature
mean_shap_values = abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({
    'feature': all_feature_names, 
    'shap_val': mean_shap_values
})
shap_df = shap_df.sort_values(by='shap_val', ascending=False)
shap_df

,feature,shap_val
39,weekend_flag,3039.872867
22,home_team_San Francisco Giants,1359.019872
36,temp,1040.277357
17,home_team_New York Yankees,883.602003
19,home_team_Philadelphia Phillies,848.566129
12,home_team_Los Angeles Dodgers,838.369523
24,home_team_St. Louis Cardinals,734.338549
11,home_team_Los Angeles Angels,711.837489
37,visiting_OPS+,695.719152
26,home_team_Texas Rangers,638.018890


In [30]:
# Remove home teams from analysis
shap_no_home_team = shap_df[~shap_df['feature'].str.startswith('home_team')]
shap_ordered = shap_no_home_team.iloc[shap_no_home_team['shap_val'].abs().argsort()[::-1]]
shap_ordered

,feature,shap_val
39,weekend_flag,3039.872867
36,temp,1040.277357
37,visiting_OPS+,695.719152
40,home_opener_flag,299.373308
29,day_night_categories_non_sunday_night,286.845126
30,day_night_categories_sunday_day,271.275248
35,bobblehead_YES,186.331805
34,fireworks_YES,71.915684
41,holiday_flag,62.503101
33,shirt_YES,58.110584


### Comparing Feature Importance Methods

In [31]:
# Create rank columns for each non-home team variable
# Join the tables and observe the effects
import warnings
warnings.filterwarnings('ignore')
shap_ordered['shap_rank'] = np.arange(1, len(shap_ordered) + 1)
coeffs_ordered['coeff_rank'] = np.arange(1, len(coeffs_ordered) + 1)
gini_ordered['gini_rank'] = np.arange(1, len(gini_ordered) + 1)
feature_importances = (
    shap_ordered
    .merge(coeffs_ordered)
    .merge(gini_ordered)
    
)

In [32]:
feature_importances[['feature', 'coeff_rank', 'gini_rank', 'shap_rank']]

,feature,coeff_rank,gini_rank,shap_rank
0,weekend_flag,2,2,1
1,temp,12,1,2
2,visiting_OPS+,10,3,3
3,home_opener_flag,7,4,4
4,day_night_categories_non_sunday_night,9,6,5
5,day_night_categories_sunday_day,6,7,6
6,bobblehead_YES,1,5,7
7,fireworks_YES,3,9,8
8,holiday_flag,11,10,9
9,shirt_YES,5,8,10


# Analysis of Results

**Takeaways:**

1) The gini importance and shapley values were quite similar despite their different methodologies. This is not entirely surprising given that they are both based on the random forest model, but the more advanced game theory approached taken when calculating shapley values only resulted in a slightly lower relative importance of promotions.

2) All three methods rated weekend games as a top-2 predictor of attendance. Accordingly, teams should always be prepared for high attendance at visiting games.

3) The linear regression model viewed bobbleheads as the most important factor to increase attendance, whereas the random forest methods found it to have only an average effect. I generally trust these more complex results based on non-linear models, as these methods likely did a better job picking up on the fact that bobbleheads are given away predominantly on weekends and highly-anticipated games. Therefore, the appropriate conclusion here is that bobbleheads have a moderate increase on attendance, whereas the other three forms of promotions are less effective.

4) The linear regression model did not view temperature as an important feature given it's non-linear relationship, but the random forest model viewed it as highly important, as expected. Perhaps squaring this term in linear regression would help better capture this relationship.

5) The non-linear methods both identified visting OPS+ as a significant predictor of attendance.

**What can we do with this information?**

1) We are able to predict attendance with an average error under 4,000. Teams can use this information to forecast demand for concessions, effectively time promotions and identify periods of high fan engagement.

2) Teams should look to capitalize revenue in games against high-powered offenses, as these are safe bets to bring in large crowds.

3) Teams should prioritize bobblehead promotions over others, as they lead to the greatest uplift in attendance.

4) Given the fact that weekend games have significantly greater demand than weekday games when accounting for other factors, ticketing departments should consider alternative strategies, bundles, and reduced prices to sell Monday-Thursday tickets.

**Next Steps:**

1) Look into building models for each individual team for more specific analysis. This could be difficult with only one season's worth of data, but is certainly possible if more data becomes available.

2) Train the model on weekday vs. weekend games and observe the relative effect of promotions in each situation. This could help the marketing department best identify games to run promotions.

3) Analyze promotions that resulted in the greatest uplift in promotions, either through Shapley Values or residual analysis, to further identify the best opportunities to run promotions.